## Tensforwflow example with two hidden layers. The MNIST dataset

### Import the relevant packages

In [93]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfsd
from datetime import datetime


### Data

In [94]:
mnist_dataset,mnist_info=tfsd.load(name='mnist',with_info=True, as_supervised=True, shuffle_files=True)

In [95]:
mnist_dataset

{'test': <_OptionsDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'train': <_OptionsDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [96]:
mnist_info.splits['test'].num_examples

10000

### Train and test datasets

In [97]:
mnist_train,mnist_test = mnist_dataset['train'],mnist_dataset['test']
num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples,tf.int64)

In [98]:
#mnist_train.element_spec
#num_validation_samples

In [99]:
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64)

In [100]:
#num_test_samples

### Normalize train and test datasets

In [101]:
def scale(image,label):
    image=tf.cast(image,tf.float32)
    image/=255. #image=image/255
    return image,label

scaled_train_and_validation_data=mnist_train.map(scale)
scaled_test_data = mnist_test.map(scale)

### Shuffle Train dataset ang separate train and validation samples

In [102]:
BUFFER_SIZE=10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

### Prepare for batching

In [103]:
validation_data = shuffled_train_and_validation_data.take(num_validation_samples) #num_validation_samples = 6000
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE= 100 #10000
train_data=train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

In [104]:
#validation_inputs.shape

### Model

In [105]:
#For 1st, 4th question

input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'), 
    #tf.keras.layers.Dense(hidden_layer_size,activation='tanh'),
    tf.keras.layers.Dense(output_size,activation='softmax')
])

In [106]:
#For 2nd, 3rd Question 
'''
input_size = 784
output_size = 10
hidden_layer_size = 70

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(output_size,activation='softmax')
])
'''

"\ninput_size = 784\noutput_size = 10\nhidden_layer_size = 70\n\nmodel = tf.keras.Sequential([\n    tf.keras.layers.Flatten(input_shape=(28,28,1)),\n    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),\n    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),\n    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),\n    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),\n    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),\n    tf.keras.layers.Dense(output_size,activation='softmax')\n])\n"

### Choose the optimizer and the loss function

In [107]:
#model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [108]:
custom_optimizer = tf.keras.optimizers.SGD(learning_rate= 0.02)
model.compile(custom_optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

### Training

In [109]:
NUM_EPOCHS=10
start = datetime.now()
model.fit(train_data,epochs=NUM_EPOCHS,validation_data=(validation_inputs, validation_targets),verbose=2)
end = datetime.now()
time_taken = end - start
print('Time: ',time_taken) 

Epoch 1/10
540/540 - 7s - loss: 0.8243 - accuracy: 0.7873 - val_loss: 0.4032 - val_accuracy: 0.8885 - 7s/epoch - 12ms/step
Epoch 2/10
540/540 - 3s - loss: 0.3545 - accuracy: 0.8999 - val_loss: 0.3141 - val_accuracy: 0.9097 - 3s/epoch - 6ms/step
Epoch 3/10
540/540 - 3s - loss: 0.2995 - accuracy: 0.9142 - val_loss: 0.2763 - val_accuracy: 0.9208 - 3s/epoch - 6ms/step
Epoch 4/10
540/540 - 3s - loss: 0.2656 - accuracy: 0.9242 - val_loss: 0.2487 - val_accuracy: 0.9260 - 3s/epoch - 6ms/step
Epoch 5/10
540/540 - 3s - loss: 0.2399 - accuracy: 0.9311 - val_loss: 0.2231 - val_accuracy: 0.9360 - 3s/epoch - 6ms/step
Epoch 6/10
540/540 - 3s - loss: 0.2198 - accuracy: 0.9373 - val_loss: 0.2008 - val_accuracy: 0.9405 - 3s/epoch - 6ms/step
Epoch 7/10
540/540 - 3s - loss: 0.2010 - accuracy: 0.9431 - val_loss: 0.1843 - val_accuracy: 0.9447 - 3s/epoch - 6ms/step
Epoch 8/10
540/540 - 3s - loss: 0.1880 - accuracy: 0.9465 - val_loss: 0.1741 - val_accuracy: 0.9488 - 3s/epoch - 6ms/step
Epoch 9/10
540/540 - 3s

In [110]:
#val_accuracy: 0.9557, val_loss: 0.1527, Train accuracy: 0.9532, accuracy: 0.9546 , Test loss: 0.1577

### Test the model

In [111]:
test_loss,test_accuracy=model.evaluate(test_data)

1/1 [==============================] - 1s 848ms/step - loss: 0.1577 - accuracy: 0.9546
